In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime, date
from ast import literal_eval

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
engine = create_engine("sqlite:///data/kickstarter.db")
df = pd.read_sql('SELECT * FROM data;', engine)
print(df.shape)
df.head()

(870114, 38)


,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,56,The Backyard will be a community garden in Lak...,"{""id"":305,""name"":""Community Gardens"",""slug"":""f...",5507,US,the United States,1487809696,"{""id"":2723718,""name"":""The Backyard"",""is_regist...",USD,$,...,the-backyard-community-garden,https://www.kickstarter.com/discover/categorie...,true,false,successful,1491789637,1,"{""web"":{""project"":""https://www.kickstarter.com...",5507.77,domestic
1,1,Modern London is home to a poverty driven ambi...,"{""id"":293,""name"":""Drama"",""slug"":""film & video/...",1,GB,the United Kingdom,1469267948,"{""id"":2084330677,""name"":""Lee Bartlett"",""is_reg...",GBP,£,...,rocks-to-stocks,https://www.kickstarter.com/discover/categorie...,false,false,failed,1471870195,1.32234378,"{""web"":{""project"":""https://www.kickstarter.com...",1.32234378,domestic
2,35,Announcing a twist of our original party game ...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1119,US,the United States,1519236804,"{""id"":1978836496,""name"":""I Can't Even! Game"",""...",USD,$,...,i-cant-even-a-party-game-expansion-pack-ice-to-go,https://www.kickstarter.com/discover/categorie...,true,false,successful,1532956446,1,"{""web"":{""project"":""https://www.kickstarter.com...",1119.0,domestic
3,37,Overstock card renovation plan,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1236,HK,Hong Kong,1566103145,"{""id"":1320915715,""name"":""yandawei"",""is_registe...",HKD,$,...,fantasy-creature-stock-modified-edition,https://www.kickstarter.com/discover/categorie...,true,false,successful,1570200330,0.12748679,"{""web"":{""project"":""https://www.kickstarter.com...",1236.49437621,domestic
4,0,Straight Up Photography is a project to bring ...,"{""id"":277,""name"":""Nature"",""slug"":""photography/...",0,US,the United States,1422146426,"{""id"":109408228,""name"":""Ivan Straight"",""is_reg...",USD,$,...,straight-up-photography,https://www.kickstarter.com/discover/categorie...,false,false,canceled,1422243394,1,"{""web"":{""project"":""https://www.kickstarter.com...",0.0,domestic


In [4]:
df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'country_displayable_name', 'created_at', 'creator',
       'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'launched_at', 'location', 'name', 'permissions', 'photo', 'pledged',
       'profile', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')

In [5]:
# get rid of the "column-headers rows"
df = df[df.created_at != "created_at"]
df.shape

(869877, 38)

In [6]:
df = df.drop(columns=['creator', 'country_displayable_name', 'currency_symbol', 'currency_trailing_code', 'current_currency', 
                            'disable_communication','friends', 'is_backing', 'is_starrable', 'is_starred', 
                            'permissions', 'photo', 'profile', 'source_url', 'urls'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 869877 entries, 0 to 870113
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   backers_count             869877 non-null  object
 1   blurb                     869877 non-null  object
 2   category                  869877 non-null  object
 3   converted_pledged_amount  869877 non-null  object
 4   country                   869877 non-null  object
 5   created_at                869877 non-null  object
 6   currency                  869877 non-null  object
 7   deadline                  869877 non-null  object
 8   fx_rate                   869877 non-null  object
 9   goal                      869877 non-null  object
 10  id                        869877 non-null  object
 11  launched_at               869877 non-null  object
 12  location                  869877 non-null  object
 13  name                      869877 non-null  object
 14  pled

In [8]:
# first looking at the ratio of successful and failed here
df.state.value_counts()

successful    513286
failed        297213
canceled       36090
live           23288
Name: state, dtype: int64

In [9]:
df['state'] = df.state.astype(str)
# Only interested in success/fail champaigns
df = df[(df.state == "successful") | (df.state == "failed")]
df.shape

(810499, 23)

In [10]:
# Converting datetime objects
df['created_at'] = df['created_at'].astype(int)
df['created_at'] = pd.to_datetime(df['created_at'],unit='s')

df['deadline'] = df['deadline'].astype(int)
df['deadline'] = pd.to_datetime(df['deadline'], unit='s')

df['launched_at'] = df['launched_at'].astype(int)
df['launched_at'] = pd.to_datetime(df['launched_at'],unit='s')

df['state_changed_at'] = df['state_changed_at'].astype(int)
df['state_changed_at'] = pd.to_datetime(df['state_changed_at'],unit='s')

In [11]:
# df['duration'] = df.deadline - df.launched_at

In [12]:
# check for duplicates
df.id.value_counts()

430731749     9
2028673237    9
1684513733    8
563445716     8
1434058810    8
             ..
750731768     1
1243134418    1
1164987667    1
1586158795    1
120722291     1
Name: id, Length: 189162, dtype: int64

In [13]:
df[df.id == '2028673237']

,backers_count,blurb,category,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,goal,...,name,pledged,slug,spotlight,staff_pick,state,state_changed_at,static_usd_rate,usd_pledged,usd_type
3661,219,Personalized pillow that fits every type of sl...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",36557,NL,2019-09-21 07:53:37,EUR,2020-10-30 09:48:30,1.21497973,10000,...,Franky's Pillow,31270,frankys-pillow,true,false,successful,2020-10-30 09:48:30,1.18473306,37046.602786199997,domestic
96273,219,Personalized pillow that fits every type of sl...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",36557,NL,2019-09-21 07:53:37,EUR,2020-10-30 09:48:30,1.21497973,10000,...,Franky's Pillow,31270,frankys-pillow,true,false,successful,2020-10-30 09:48:30,1.18473306,37046.602786199997,domestic
162383,219,Personalized pillow that fits every type of sl...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",36557,NL,2019-09-21 07:53:37,EUR,2020-10-30 09:48:30,1.21497973,10000,...,Franky's Pillow,31270,frankys-pillow,true,false,successful,2020-10-30 09:48:30,1.18473306,37046.602786199997,domestic
233190,219,Personalized pillow that fits every type of sl...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",36557,NL,2019-09-21 07:53:37,EUR,2020-10-30 09:48:30,1.21220937,10000,...,Franky's Pillow,31270,frankys-pillow,true,false,successful,2020-10-30 09:48:30,1.18473306,37046.602786199997,domestic
331971,219,Personalized pillow that fits every type of sl...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",36557,NL,2019-09-21 07:53:37,EUR,2020-10-30 09:48:30,1.21220937,10000,...,Franky's Pillow,31270,frankys-pillow,true,false,successful,2020-10-30 09:48:30,1.18473306,37046.602786199997,international
468787,219,Personalized pillow that fits every type of sl...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",36557,NL,2019-09-21 07:53:37,EUR,2020-10-30 09:48:30,1.19592048,10000,...,Franky's Pillow,31270,frankys-pillow,true,false,successful,2020-10-30 09:48:30,1.18473306,37046.602786199997,international
581029,219,Personalized pillow that fits every type of sl...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",36557,NL,2019-09-21 07:53:37,EUR,2020-10-30 09:48:30,1.19592048,10000,...,Franky's Pillow,31270,frankys-pillow,true,false,successful,2020-10-30 09:48:30,1.18473306,37046.602786199997,international
738967,219,Personalized pillow that fits every type of sl...,"{""id"":28,""name"":""Product Design"",""analytics_na...",36557,NL,2019-09-21 07:53:37,EUR,2020-10-30 09:48:30,1.19752161,10000,...,Franky's Pillow,31270,frankys-pillow,true,false,successful,2020-10-30 09:48:30,1.18473306,37046.602786199997,international
856122,219,Personalized pillow that fits every type of sl...,"{""id"":28,""name"":""Product Design"",""analytics_na...",36557,NL,2019-09-21 07:53:37,EUR,2020-10-30 09:48:30,1.19752161,10000,...,Franky's Pillow,31270,frankys-pillow,true,false,successful,2020-10-30 09:48:30,1.18473306,37046.602786199997,international


In [14]:
df.drop_duplicates(subset=['id', 'state'], inplace=True)
df.shape

(189162, 23)

In [15]:
# Categories - Main category and sub_category
c = df.category.apply(literal_eval)
main_category = []
for i, d in enumerate(c):
    try:
        main_category.append(d['parent_name'])
    except:
        main_category.append(np.nan)
df['main_category'] = main_category
df.main_category.value_counts()

Film & Video    25683
Music           25432
Technology      19254
Art             19232
Publishing      18814
Food            14321
Games           12690
Fashion         10083
Comics           7192
Design           6892
Photography      5894
Crafts           5181
Theater          4708
Journalism       3402
Dance            1716
Name: main_category, dtype: int64

In [16]:
sub_category = []
for i, name in enumerate(d['name'] for d in c):
    sub_category.append(name)
df['sub_category'] = sub_category
df.sub_category.value_counts()

Web               3805
Tabletop Games    3275
Comedy            2925
Comic Books       2602
Product Design    2600
                  ... 
Quilts              87
Residencies         85
Letterpress         84
Chiptune            50
Taxidermy           10
Name: sub_category, Length: 161, dtype: int64

In [17]:
# Country - needs to condense the list
df.country.value_counts()

US    130527
GB     21365
CA      8829
AU      4399
DE      3487
FR      2826
MX      2770
IT      2467
ES      2201
NL      1648
SE      1380
HK      1322
DK       851
NZ       826
SG       767
CH       685
BE       587
IE       580
JP       554
AT       470
NO       449
LU        62
PL        59
GR        35
SI        16
Name: country, dtype: int64

In [18]:
dic = {"DK": "OTHER", "NZ": "OTHER", "SG": "OTHER", "CH": "OTHER", "BE": "OTHER", "IE": "OTHER",
      "JP": "OTHER", "AT": "OTHER", "NO": "OTHER", "LU": "OTHER", "PL": "OTHER", "GR": "OTHER",
      "SI": "OTHER"}
df = df.replace({"country": dic})
df.country.value_counts()

US       130527
GB        21365
CA         8829
OTHER      5941
AU         4399
DE         3487
FR         2826
MX         2770
IT         2467
ES         2201
NL         1648
SE         1380
HK         1322
Name: country, dtype: int64

In [19]:
# Currency - needs to convert all goal values to USD
df.currency.value_counts()

USD    130527
GBP     21365
EUR     14516
CAD      8829
AUD      4399
MXN      2770
SEK      1335
HKD      1322
NZD       826
DKK       824
SGD       767
CHF       666
JPY       554
NOK       439
PLN        23
Name: currency, dtype: int64

In [20]:
df['goal'] = df.goal.astype(float)
df['fx_rate'] = df.fx_rate.astype(float)
df['goal_usd'] = df.goal * df.fx_rate
df.head()

,backers_count,blurb,category,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,goal,...,spotlight,staff_pick,state,state_changed_at,static_usd_rate,usd_pledged,usd_type,main_category,sub_category,goal_usd
0,56,The Backyard will be a community garden in Lak...,"{""id"":305,""name"":""Community Gardens"",""slug"":""f...",5507,US,2017-02-23 00:28:16,USD,2017-04-10 02:00:36,1.000000,5227.0,...,true,false,successful,2017-04-10 02:00:37,1,5507.77,domestic,Food,Community Gardens,5227.00000
1,1,Modern London is home to a poverty driven ambi...,"{""id"":293,""name"":""Drama"",""slug"":""film & video/...",1,GB,2016-07-23 09:59:08,GBP,2016-08-22 12:49:55,1.367950,35000.0,...,false,false,failed,2016-08-22 12:49:55,1.32234378,1.32234378,domestic,Film & Video,Drama,47878.24160
2,35,Announcing a twist of our original party game ...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1119,US,2018-02-21 18:13:24,USD,2018-07-30 13:14:04,1.000000,1000.0,...,true,false,successful,2018-07-30 13:14:06,1,1119.0,domestic,Games,Playing Cards,1000.00000
3,37,Overstock card renovation plan,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1236,HK,2019-08-18 04:39:05,HKD,2019-10-04 14:45:30,0.128969,8000.0,...,true,false,successful,2019-10-04 14:45:30,0.12748679,1236.49437621,domestic,Games,Playing Cards,1031.74888
5,14,I want to learn how to dye fabric! Follow me a...,"{""id"":289,""name"":""Textiles"",""slug"":""art/textil...",766,US,2018-01-06 20:19:17,USD,2018-02-10 02:29:02,1.000000,550.0,...,true,false,successful,2018-02-10 02:29:04,1,766.0,domestic,Art,Textiles,550.00000


In [21]:
df.usd_type.value_counts()

domestic         183869
international      5083
null                210
Name: usd_type, dtype: int64

In [22]:
# checking for NaN
df.isna().sum()

backers_count                  0
blurb                          0
category                       0
converted_pledged_amount       0
country                        0
created_at                     0
currency                       0
deadline                       0
fx_rate                        0
goal                           0
id                             0
launched_at                    0
location                       0
name                           0
pledged                        0
slug                           0
spotlight                      0
staff_pick                     0
state                          0
state_changed_at               0
static_usd_rate                0
usd_pledged                    0
usd_type                       0
main_category               8668
sub_category                   0
goal_usd                       0
dtype: int64

In [23]:
# fill main_category NaN with sub_category 
df[df.main_category.isna()]['sub_category'].value_counts()

Dance           1378
Art             1259
Crafts          1130
Music            790
Journalism       723
Food             574
Theater          555
Photography      456
Comics           364
Technology       363
Publishing       336
Design           249
Film & Video     244
Fashion          146
Games            101
Name: sub_category, dtype: int64

In [24]:
df.main_category.fillna(df.sub_category, inplace=True)
df.isna().sum()

backers_count               0
blurb                       0
category                    0
converted_pledged_amount    0
country                     0
created_at                  0
currency                    0
deadline                    0
fx_rate                     0
goal                        0
id                          0
launched_at                 0
location                    0
name                        0
pledged                     0
slug                        0
spotlight                   0
staff_pick                  0
state                       0
state_changed_at            0
static_usd_rate             0
usd_pledged                 0
usd_type                    0
main_category               0
sub_category                0
goal_usd                    0
dtype: int64

In [26]:
kickstarter = df[['id', 'goal_usd', 'country', 'created_at', 'deadline',  'launched_at', 'state_changed_at',
                  'spotlight','staff_pick', 'main_category', 'sub_category', 'blurb', 'state']]
kickstarter.head()

,id,goal_usd,country,created_at,deadline,launched_at,state_changed_at,spotlight,staff_pick,main_category,sub_category,blurb,state
0,1560919980,5227.00000,US,2017-02-23 00:28:16,2017-04-10 02:00:36,2017-03-11 03:00:36,2017-04-10 02:00:37,true,false,Food,Community Gardens,The Backyard will be a community garden in Lak...,successful
1,1441173176,47878.24160,GB,2016-07-23 09:59:08,2016-08-22 12:49:55,2016-07-23 12:49:55,2016-08-22 12:49:55,false,false,Film & Video,Drama,Modern London is home to a poverty driven ambi...,failed
2,1907302929,1000.00000,US,2018-02-21 18:13:24,2018-07-30 13:14:04,2018-06-30 13:14:04,2018-07-30 13:14:06,true,false,Games,Playing Cards,Announcing a twist of our original party game ...,successful
3,520777940,1031.74888,HK,2019-08-18 04:39:05,2019-10-04 14:45:30,2019-09-04 14:45:30,2019-10-04 14:45:30,true,false,Games,Playing Cards,Overstock card renovation plan,successful
5,2087823414,550.00000,US,2018-01-06 20:19:17,2018-02-10 02:29:02,2018-01-11 02:29:02,2018-02-10 02:29:04,true,false,Art,Textiles,I want to learn how to dye fabric! Follow me a...,successful


In [27]:
kickstarter.to_csv("./data/kickstarter.csv", index= False)